In [3]:
%defaultDatasource jdbc:h2:mem:db

UsageError: Line magic function `%defaultDatasource` not found.


In [16]:
DROP TABLE IF EXISTS Covid;

CREATE TABLE Covid (
  Confirmed INTEGER,
  Deaths INTEGER,
  Recovered INTEGER,
  Active INTEGER,
  Data VARCHAR (20) NOT NULL,
  PRIMARY KEY(Data)
) AS SELECT
    Confirmed,
    Deaths,
    Recovered,
    Active,
    Data
FROM CSVREAD('covid.csv');

In [8]:
SELECT * FROM Covid

In [41]:
DROP TABLE IF EXISTS Temperatura;

CREATE TABLE Temperatura (
  Maximo INTEGER,
  Minimo INTEGER,
  Media INTEGER,
  Data DATE,
  PRIMARY KEY(Data)
) AS SELECT
    maxtempC,
    mintempC,
    avgtempC,
    Data
FROM CSVREAD('weather.csv');

In [42]:
SELECT * FROM Temperatura

* Selecionando um intervalo de datas a partir do formato atual da tabela Covid

In [21]:
SELECT * FROM Covid
WHERE Data LIKE '%-03-%' AND RIGHT(LEFT(Data,10),2) < 21 AND RIGHT(LEFT(Data,10),2) > 09

* Por conta da dificuldade ao selecionar datas, criamos uma tabela nova, alterando a coluna "Data" para um formato padrão de Data

In [32]:
DROP TABLE IF EXISTS Covid_Data;

CREATE TABLE Covid_Data (
  Confirmed INTEGER,
  Deaths INTEGER,
  Recovered INTEGER,
  Active INTEGER,
  Data DATE,
  PRIMARY KEY(Data)
) AS SELECT
    Confirmed,
    Deaths,
    Recovered,
    Active,
    LEFT(Data,10)
FROM Covid;

SELECT * FROM Covid_Data

* Junção de ambas as tabelas a partir da data

In [45]:
DROP VIEW IF EXISTS Covid_Temp;

CREATE VIEW Covid_Temp AS
SELECT 
C.Confirmed, 
(C.Confirmed - LAG(C.Confirmed))/LAG(C.Confirmed) OVER (ORDER BY T.Data) AS ConfirmedIncrease,
C.Deaths, 
(C.Deaths - LAG(C.Deaths))/LAG(C.Deaths) OVER (ORDER BY T.Data) AS DeathsIncrease,
C.Recovered, 
T.Maximo, 
T.Minimo,
T.Media,
(T.Media - LAG(T.Media))/LAG(T.Media) OVER (ORDER BY T.Data) AS AverageTemperatureIncrease,
T.Data, 
FROM Covid_Data AS C, Temperatura AS T
WHERE C.Data = T.Data;
ORDER BY T.Data

SELECT * FROM Covid_Temp